## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [ ]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [ ]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [ ]:
# Se establece qué columnas se eliminan
col_eliminar_creditos = ["completitud_fila", "situacion_vivienda_ok", "objetivo_credito_ok", "estado_credito_ok", "falta_pago_ok"]
col_eliminar_tarjetas = ["nivel_tarjeta"]

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [ ]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Limpieza de los datos

In [ ]:
'''
Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente 
mayor al 50% sus ingresos deberán ser mayores a 20.000.
'''

#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,ok


In [ ]:
'''
Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos
del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos de los que sea propietario de su vivienda.
'''

#Se puede definir una función para aplicar los cálculos
def regla_duracion_credito(row):
    duracion_credito = row.duracion_credito
    ingresos = row.ingresos
    situacion_vivienda=row.situacion_vivienda
    importe_solicitado=row.importe_solicitado
    pct_ingresos=row.pct_ingreso
    if duracion_credito <= 2:
        if situacion_vivienda != 'PROPIA':
            if pct_ingresos>0.6:
            # Es un error, no cumple la regla definida
                return 'err'
            else:
                return 'ok'
        else:
            #no se aplica la regla para propietarios
            return 'ok'
    else:
        #no se aplica la regla para duraciones mayores a la mínima
        return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_credito = df_integrado.apply(lambda row: regla_duracion_credito(row), axis=1).rename("regla_duracion_credito")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_duracion_credito], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_credito
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,ok,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,ok,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,ok,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,ok,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,ok,ok


In [ ]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtrado: {df_creditos.shape[0]}")
temp_1 = df_creditos[df_creditos['edad'] < 90]

temp_2 = temp_1[temp_1['antiguedad_empleado'] <= 50] 

temp_3 = temp_2[temp_2['regla_pct_ingresos'] == 'ok']  # Regla 1

temp_4 = temp_3[temp_3['regla_duracion_credito'] == 'ok']  # Regla 2

print(f"Filas después del filtrado: {temp_4.shape[0]}")

Filas antes del filtrado: 10127
Filas después del filtrado: 9765


Integración de datos

In [ ]:
df_integrado = pd.merge(temp_4, df_tarjetas, on='id_cliente', how='inner') # Unir los dataset sobre los que hayamos aplicado los filtros
coincidencias = df_integrado.shape[0]

print(f"Cantidad de filas del dataset integrado: {coincidencias}")
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Transformación de atributos

In [ ]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')
df_final = pd.concat([estado_civil_N, df_integrado], axis=1)
df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

Exportación de metadatos